In [1]:
#TODOs 
# Add the case when we DO have yesterday's scrapped companies but we don't have today's scrapped companies cause the day just changed
# Add Data Origin (Linkedin in this case)

In [2]:
import pandas as pd
import re
import random
from datetime import date, timedelta

import selenium
from bs4 import BeautifulSoup as soup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from urllib.request import urlopen as ureq
import urllib

from bs4 import BeautifulSoup as soup

from warnings import warn

# Scraping for job in AU

In [3]:
url = 'https://au.linkedin.com/jobs/data-analyst-jobs-sydney-nsw?keywords=Data%20Analyst&location=Sydney%2C%20New%20South%20Wales%2C%20Australia&locationId=&geoId=104769905&sortBy=DD&f_TPR=r86400&distance=25&position=1&pageNum=0'

### Opening up connection and getting HTML info from the webpage

In [4]:
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

In [5]:
# New jobs posted within set timeframe
new_jobs_posted = int(driver.find_element_by_xpath('//span[@class="results-context-header__job-count"]').text) - (int(driver.find_element_by_xpath('//span[@class="results-context-header__job-count"]').text)*0.2)
new_jobs_posted

96.8

In [6]:
contenedores = []

while len(contenedores) < new_jobs_posted:
    try:
        driver.find_element_by_xpath('//button[@class="infinite-scroller__show-more-button infinite-scroller__show-more-button--visible"]').click()
    except:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        contenedores = driver.find_elements_by_xpath('//div[@class="base-card base-card--link base-search-card base-search-card--link job-search-card"]')
        sleep(random.uniform(4.0,8.0))

In [7]:
len(contenedores)

98

In [8]:
contenedores[0]

<selenium.webdriver.remote.webelement.WebElement (session="f1722af644aa34f0170063325c4e409a", element="750bd620-152d-49a1-962c-6dec8f975791")>

In [9]:
contenedores[0].text

'Business Analyst\nWestpac Group\nSydney, New South Wales, Australia\nBe an early applicant\n1 hour ago'

### Retrieving all the DIVs that contain job postings

In [10]:
#Job titles
job_titles=[]
company_name=[]
job_location=[]
date_scraped=[]
links=[]
job_industry=[]
job_desc=[]
job_type=[]
job_seniority=[]

counter=1

#Check for the app to stop if the job was already scrapped
df = pd.read_csv('Linkedin_data_info_au.csv', engine='python')

In [11]:
try:
    last_index_in_csv = int(df.iloc[-1]['Unnamed: 0'])
except:
    pass

In [12]:
(df['Date Scraped']).replace(to_replace=r'\/',value='-',regex=True,inplace=True)

In [13]:
# Try to get jobs posted today and yesterday. If yesteday cannot be found, get the jobs added today. If neither can be found, instantiate empty variables
try:
    job_title_check = [df[df['Date Scraped']==str(date.today().strftime('%d-%m-%Y'))]['Job Title'].iloc[0],df[df['Date Scraped']==(date.today() - timedelta(days=1)).strftime('%d-%m-%Y')]['Job Title'].iloc[0]]
    company_name_check = [df[df['Date Scraped']==str(date.today().strftime('%d-%m-%Y'))]['Company Name'].iloc[0], df[df['Date Scraped']==(date.today() - timedelta(days=1)).strftime('%d-%m-%Y')]['Company Name'].iloc[0]]
except: 
    try:
        job_title_check = df[df['Date Scraped']==str(date.today().strftime('%d-%m-%Y'))]['Job Title'].iloc[0]
        company_name_check = df[df['Date Scraped']==str(date.today().strftime('%d-%m-%Y'))]['Company Name'].iloc[0]
    except:
        try:
            job_title_check = df[df['Date Scraped']==(date.today() - timedelta(days=1)).strftime('%d-%m-%Y')]['Job Title'].iloc[0]
            company_name_check = df[df['Date Scraped']==(date.today() - timedelta(days=1)).strftime('%d-%m-%Y')]['Company Name'].iloc[0]
        except:
            job_title_check=''
            company_name_check=''

In [14]:
job_title_check

''

In [15]:
company_name_check

''

In [16]:
#Scraping the data
for job_post in contenedores:
    job_title=job_post.find_element_by_xpath('.//h3[@class="base-search-card__title"]').text
    job_company_name=job_post.find_element_by_xpath('.//h4[@class="base-search-card__subtitle"]').text
    if job_title==job_title_check and job_company_name==company_name_check:
        break
    #Job title
    job_titles.append(job_title)
    #Company name
    company_name.append(job_company_name)
    #Job Location
    job_location.append(job_post.find_element_by_xpath('.//span[@class="job-search-card__location"]').text)
    #Date scraped
    date_scraped.append(str(date.today()))
    #Job link
    links.append(job_post.find_element_by_xpath('.//a[@class="base-card__full-link"]').get_attribute('href'))
    #Retrieve Job Type, Industry and Seniority
    driver.find_elements_by_xpath('//a[@class="base-card__full-link"]')[counter].click()
    sleep(random.uniform(2.1,5.6))
    job_desc.append(driver.find_element_by_xpath('.//section[@class="description"]/div/section/div').text.strip())
    
    try:
        job_type.append(driver.find_element_by_xpath('.//ul[@class="description__job-criteria-list"]/li[2]').text.strip())
        job_industry.append(driver.find_element_by_xpath('.//ul[@class="description__job-criteria-list"]/li[4]').text)
        job_seniority.append(driver.find_element_by_xpath('.//ul[@class="description__job-criteria-list"]/li[1]').text.strip())
    except:
        job_industry.append('NA')
        job_type.append('NA')
        job_seniority.append('NA')

    counter=counter+1
    sleep(random.uniform(1.0,3.5))

In [17]:
contenedores[0].find_element_by_xpath('.//h3[@class="base-search-card__title"]').text

'Business Analyst'

In [18]:
print('Job titles ----')
print(len(job_titles))    
print(job_titles[0:2])
print('Job company ----')
print(len(company_name))
print(company_name[0:2])
print('Job Location ----')
print(len(job_location))
print(job_location[0:2])
print('Date scraped ----')
print(len(date_scraped))
print(date_scraped[0:2])
print('Job links ----')
print(len(links))
print(links[0:2])
print('Job type ----')
print(len(job_type))
print(job_type[0:2])
print('Job industry ----')
print(job_industry[0:2])
print('Job seniority ----')
print(job_seniority[0:2])

Job titles ----
98
['Business Analyst', 'Senior Associate, Customer Remediation Analytics']
Job company ----
98
['Westpac Group', 'Westpac Institutional Bank']
Job Location ----
98
['Sydney, New South Wales, Australia', 'Sydney, New South Wales, Australia']
Date scraped ----
98
['2021-08-27', '2021-08-27']
Job links ----
98
['https://au.linkedin.com/jobs/view/business-analyst-at-westpac-group-2669985490?refId=KrVGxPRtRwDQXXR1DqUK%2FA%3D%3D&trackingId=2%2FXFwKS84gaW4Dnv1l5Fvg%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://au.linkedin.com/jobs/view/senior-associate-customer-remediation-analytics-at-westpac-institutional-bank-2693196354?refId=KrVGxPRtRwDQXXR1DqUK%2FA%3D%3D&trackingId=1cc0n5ycVBSiPgFnBYGlgw%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card']
Job type ----
98
['Employment type\nFull-time', 'Employment type\nFull-time']
Job industry ----
['Industries\nBanking, Financial Services, and Investment Banking', 'Industries\nBanking,

### 

In [19]:
dataframe_structure = {'Job Title': job_titles, 'Company Name':company_name, 'Location':job_location, 'Date Scraped':date_scraped, 'Job URL':links, 'Job Industry':job_industry, 'Job Type':job_type, 'Job Seniority':job_seniority, 'Job Description':job_desc}

In [20]:
try:
    last_index_in_csv_1 = last_index_in_csv+1
except:
    last_index_in_csv_1= 0
df = pd.DataFrame(dataframe_structure)
df.index += last_index_in_csv_1

In [21]:
df['Job Industry'] = df['Job Industry'].replace(r'Industries\n','',regex=True)
df['Job Seniority'] = df['Job Seniority'].replace(r'Seniority level\n','',regex=True)
df['Job Description'] = df['Job Description'].replace(r'Job Description\n','',regex=True).replace(r'\n',' ',regex=True)
df['Job Type'] = df['Job Type'].replace(r'Employment Type\n','',regex=True)

In [22]:
df.to_csv('Linkedin_data_info_au.csv',mode='a',header=False)

# Scraping for job in US

In [23]:
url = 'https://au.linkedin.com/jobs/search?keywords=Data%20Analyst&location=United%20States&locationId=&geoId=103644278&sortBy=DD&f_TPR=r86400&position=1&pageNum=0'

### Opening up connection and getting HTML info from the webpage

In [24]:
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

In [25]:
try:
    new_jobs_posted = re.findall('\d+',driver.find_element_by_xpath('//span[@class="results-context-header__job-count"]').text)
    new_jobs_posted = int("".join(new_jobs_posted))
except:
    new_jobs_posted = int(driver.find_element_by_xpath('//span[@class="results-context-header__job-count"]').text) - (int(driver.find_element_by_xpath('//span[@class="results-context-header__job-count"]').text)*0.2)

if new_jobs_posted > 400:
    new_jobs_posted = 400

In [26]:
contenedores = []

while len(contenedores) < new_jobs_posted:
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.find_element_by_xpath('//button[@class="infinite-scroller__show-more-button infinite-scroller__show-more-button--visible"]').click()
        contenedores = driver.find_elements_by_xpath('//div[@class="base-card base-card--link base-search-card base-search-card--link job-search-card"]')
        sleep(random.uniform(1.7,5.3))
    except:
        driver.execute_script("window.scrollTo(0, 200)")
        sleep(random.uniform(0.7,1.3))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        contenedores = driver.find_elements_by_xpath('//div[@class="base-card base-card--link base-search-card base-search-card--link job-search-card"]')
        sleep(random.uniform(4.0,8.0))

In [27]:
len(contenedores)

424

In [28]:
new_jobs_posted

400

### Retrieving all the DIVs that contain job postings

In [29]:
#Job titles
job_titles=[]
company_name=[]
job_location=[]
date_scraped=[]
links=[]
job_industry=[]
job_desc=[]
job_type=[]
job_seniority=[]

counter=1

#Check for the app to stop if the job was already scrapped
try:
    df = pd.read_csv('Linkedin_data_info_us.csv', engine='python')
    last_index_in_csv = int(df.iloc[-1]['Unnamed: 0']) 
except:
    pass

In [30]:
try: 
    (df['Date Scraped']).replace(to_replace=r'\/',value='-',regex=True,inplace=True)
except: 
    pass

In [31]:
# Try to get jobs posted today and yesterday. If yesteday cannot be found, get the jobs added today. If neither can be found, instantiate empty variables
try:
    job_title_check = [df[df['Date Scraped']==str(date.today().strftime('%d-%m-%Y'))]['Job Title'].iloc[0],df[df['Date Scraped']==(date.today() - timedelta(days=1)).strftime('%d-%m-%Y')]['Job Title'].iloc[0]]
    company_name_check = [df[df['Date Scraped']==str(date.today().strftime('%d-%m-%Y'))]['Company Name'].iloc[0], df[df['Date Scraped']==(date.today() - timedelta(days=1)).strftime('%d-%m-%Y')]['Company Name'].iloc[0]]
except: 
    try:
        job_title_check = df[df['Date Scraped']==str(date.today().strftime('%d-%m-%Y'))]['Job Title'].iloc[0]
        company_name_check = df[df['Date Scraped']==str(date.today().strftime('%d-%m-%Y'))]['Company Name'].iloc[0]
    except:
        try:
            job_title_check = df[df['Date Scraped']==(date.today() - timedelta(days=1)).strftime('%d-%m-%Y')]['Job Title'].iloc[0]
            company_name_check = df[df['Date Scraped']==(date.today() - timedelta(days=1)).strftime('%d-%m-%Y')]['Company Name'].iloc[0]
        except:
            job_title_check=''
            company_name_check=''

In [32]:
job_title_check

''

In [33]:
company_name_check

''

In [34]:
#Scraping the data
for job_post in contenedores:
    job_title=job_post.find_element_by_xpath('.//h3[@class="base-search-card__title"]').text
    job_company_name=job_post.find_element_by_xpath('.//h4[@class="base-search-card__subtitle"]').text
    if job_title==job_title_check and job_company_name==company_name_check:
        break
    #Job title
    job_titles.append(job_title)
    #Company name
    company_name.append(job_company_name)
    #Job Location
    job_location.append(job_post.find_element_by_xpath('.//span[@class="job-search-card__location"]').text)
    #Date scraped
    date_scraped.append(str(date.today()))
    #Job link
    links.append(job_post.find_element_by_xpath('.//a[@class="base-card__full-link"]').get_attribute('href'))
    #Retrieve Job Type, Industry and Seniority
    driver.find_elements_by_xpath('//a[@class="base-card__full-link"]')[counter].click()
    sleep(random.uniform(2.1,5.6))
    job_desc.append(driver.find_element_by_xpath('.//section[@class="description"]/div/section/div').text.strip())
    
    try:
        job_type.append(driver.find_element_by_xpath('.//ul[@class="description__job-criteria-list"]/li[2]').text.strip())
        job_industry.append(driver.find_element_by_xpath('.//ul[@class="description__job-criteria-list"]/li[4]').text)
        job_seniority.append(driver.find_element_by_xpath('.//ul[@class="description__job-criteria-list"]/li[1]').text.strip())
    except:
        job_industry.append('NA')
        job_type.append('NA')
        job_seniority.append('NA')

    counter=counter+1
    sleep(random.uniform(1.0,3.5))

In [35]:
print('Job titles ----')
print(len(job_titles))    
print(job_titles[0:2])
print('Job company ----')
print(len(company_name))
print(company_name[0:2])
print('Job Location ----')
print(len(job_location))
print(job_location[0:2])
print('Date scraped ----')
print(len(date_scraped))
print(date_scraped[0:2])
print('Job links ----')
print(len(links))
print(links[0:2])
print('Job type ----')
print(len(job_type))
print(job_type[0:2])
print('Job industry ----')
print(job_industry[0:2])
print('Job seniority ----')
print(job_seniority[0:2])

Job titles ----
424
['Software Engineer, Mobile Developer Productivity', 'Data Scientist, Credit Risk']
Job company ----
424
['Stripe', 'Stripe']
Job Location ----
424
['United States', 'Seattle, WA']
Date scraped ----
424
['2021-08-27', '2021-08-27']
Job links ----
424
['https://www.linkedin.com/jobs/view/software-engineer-mobile-developer-productivity-at-stripe-2431522021?refId=rg%2FwqksdU1%2FGCrlPCaX2Cg%3D%3D&trackingId=AQZJSEiaDcdXwzcwAXDm2w%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://www.linkedin.com/jobs/view/data-scientist-credit-risk-at-stripe-2607184408?refId=rg%2FwqksdU1%2FGCrlPCaX2Cg%3D%3D&trackingId=%2BjiMAo5o3g4MDN0wTLlZxw%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card']
Job type ----
424
['Employment type\nFull-time', 'Employment type\nFull-time']
Job industry ----
['Industries\nComputer Software, Financial Services, and Internet', 'Industries\nComputer Software, Financial Services, and Internet']
Job seniority ----


In [36]:
dataframe_structure = {'Job Title': job_titles, 'Company Name':company_name, 'Location':job_location, 'Date Scraped':date_scraped, 'Job URL':links, 'Job Industry':job_industry, 'Job Type':job_type, 'Job Seniority':job_seniority, 'Job Description':job_desc}

In [37]:
try:
    last_index_in_csv_1 = last_index_in_csv+1
except:
    last_index_in_csv_1 = 0
df = pd.DataFrame(dataframe_structure)
df.index += last_index_in_csv_1

In [38]:
df['Job Industry'] = df['Job Industry'].replace(r'Industries\n','',regex=True)
df['Job Seniority'] = df['Job Seniority'].replace(r'Seniority level\n','',regex=True)
df['Job Description'] = df['Job Description'].replace(r'Job Description\n','',regex=True).replace(r'\n',' ',regex=True)
df['Job Type'] = df['Job Type'].replace(r'Employment Type\n','',regex=True)

In [39]:
df.to_csv('Linkedin_data_info_us.csv',mode='a',header=False)

# Scraping for job in Spain

In [40]:
url = 'https://au.linkedin.com/jobs/search?keywords=Data%20Analyst&location=Spain&locationId=&geoId=105646813&sortBy=DD&f_TPR=r86400&position=1&pageNum=0'

### Opening up connection and getting HTML info from the webpage

In [41]:
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

In [42]:
# New jobs posted within set timeframe
new_jobs_posted = int(driver.find_element_by_xpath('//span[@class="results-context-header__job-count"]').text) - (int(driver.find_element_by_xpath('//span[@class="results-context-header__job-count"]').text)*0.2)
new_jobs_posted

92.8

In [43]:
contenedores = []

while len(contenedores) < new_jobs_posted:
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.find_element_by_xpath('//button[@class="infinite-scroller__show-more-button infinite-scroller__show-more-button--visible"]').click()
        contenedores = driver.find_elements_by_xpath('//div[@class="base-card base-card--link base-search-card base-search-card--link job-search-card"]')
        sleep(random.uniform(1.7,5.3))
    except:
        driver.execute_script("window.scrollTo(0, 200)")
        sleep(random.uniform(0.7,1.3))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        contenedores = driver.find_elements_by_xpath('//div[@class="base-card base-card--link base-search-card base-search-card--link job-search-card"]')
        sleep(random.uniform(4.0,8.0))

In [44]:
len(contenedores)

95

In [45]:
contenedores[0]

<selenium.webdriver.remote.webelement.WebElement (session="7292becbcb4618e57ade671df81a37a3", element="e5eb0aa9-c5fd-47e8-9b7f-e256325a5b85")>

In [46]:
contenedores[0].text

'Quality & Statistics Analyst\nKantar\nSant Cugat del Vallès, Catalonia, Spain\nBe an early applicant\n2 minutes ago'

### Retrieving all the DIVs that contain job postings

In [47]:
#Job titles
job_titles=[]
company_name=[]
job_location=[]
date_scraped=[]
links=[]
job_industry=[]
job_desc=[]
job_type=[]
job_seniority=[]

counter=1

#Check for the app to stop if the job was already scrapped
try:
    df = pd.read_csv('Linkedin_data_info_sp.csv', engine='python')
    last_index_in_csv = int(df.iloc[-1]['Unnamed: 0'])
except:
    pass

In [48]:
try:
    (df['Date Scraped']).replace(to_replace=r'\/',value='-',regex=True,inplace=True)
except:
    pass

In [49]:
# Try to get jobs posted today and yesterday. If yesteday cannot be found, get the jobs added today. If neither can be found, instantiate empty variables
try:
    job_title_check = [df[df['Date Scraped']==str(date.today().strftime('%d-%m-%Y'))]['Job Title'].iloc[0],df[df['Date Scraped']==(date.today() - timedelta(days=1)).strftime('%d-%m-%Y')]['Job Title'].iloc[0]]
    company_name_check = [df[df['Date Scraped']==str(date.today().strftime('%d-%m-%Y'))]['Company Name'].iloc[0], df[df['Date Scraped']==(date.today() - timedelta(days=1)).strftime('%d-%m-%Y')]['Company Name'].iloc[0]]
except: 
    try:
        job_title_check = df[df['Date Scraped']==str(date.today().strftime('%d-%m-%Y'))]['Job Title'].iloc[0]
        company_name_check = df[df['Date Scraped']==str(date.today().strftime('%d-%m-%Y'))]['Company Name'].iloc[0]
    except:
        try:
            job_title_check = df[df['Date Scraped']==(date.today() - timedelta(days=1)).strftime('%d-%m-%Y')]['Job Title'].iloc[0]
            company_name_check = df[df['Date Scraped']==(date.today() - timedelta(days=1)).strftime('%d-%m-%Y')]['Company Name'].iloc[0]
        except:
            job_title_check=''
            company_name_check=''

In [50]:
job_title_check

''

In [51]:
company_name_check

''

In [52]:
#Scraping the data
for job_post in contenedores:
    job_title=job_post.find_element_by_xpath('.//h3[@class="base-search-card__title"]').text
    job_company_name=job_post.find_element_by_xpath('.//h4[@class="base-search-card__subtitle"]').text
    if job_title==job_title_check and job_company_name==company_name_check:
        break
    #Job title
    job_titles.append(job_title)
    #Company name
    company_name.append(job_company_name)
    #Job Location
    job_location.append(job_post.find_element_by_xpath('.//span[@class="job-search-card__location"]').text)
    #Date scraped
    date_scraped.append(str(date.today()))
    #Job link
    links.append(job_post.find_element_by_xpath('.//a[@class="base-card__full-link"]').get_attribute('href'))
    #Retrieve Job Type, Industry and Seniority
    driver.find_elements_by_xpath('//a[@class="base-card__full-link"]')[counter].click()
    sleep(random.uniform(2.1,5.6))
    job_desc.append(driver.find_element_by_xpath('.//section[@class="description"]/div/section/div').text.strip())
    
    try:
        job_type.append(driver.find_element_by_xpath('.//ul[@class="description__job-criteria-list"]/li[2]').text.strip())
        job_industry.append(driver.find_element_by_xpath('.//ul[@class="description__job-criteria-list"]/li[4]').text)
        job_seniority.append(driver.find_element_by_xpath('.//ul[@class="description__job-criteria-list"]/li[1]').text.strip())
    except:
        job_industry.append('NA')
        job_type.append('NA')
        job_seniority.append('NA')

    counter=counter+1
    sleep(random.uniform(1.0,3.5))

In [53]:
print('Job titles ----')
print(len(job_titles))    
print(job_titles[0:2])
print('Job company ----')
print(len(company_name))
print(company_name[0:2])
print('Job Location ----')
print(len(job_location))
print(job_location[0:2])
print('Date scraped ----')
print(len(date_scraped))
print(date_scraped[0:2])
print('Job links ----')
print(len(links))
print(links[0:2])
print('Job type ----')
print(len(job_type))
print(job_type[0:2])
print('Job industry ----')
print(job_industry[0:2])
print('Job seniority ----')
print(job_seniority[0:2])

Job titles ----
95
['Quality & Statistics Analyst', 'Data Engineer_Scala. Inglés Alto']
Job company ----
95
['Kantar', 'CGI']
Job Location ----
95
['Sant Cugat del Vallès, Catalonia, Spain', 'Greater Madrid Metropolitan Area']
Date scraped ----
95
['2021-08-27', '2021-08-27']
Job links ----
95
['https://es.linkedin.com/jobs/view/quality-statistics-analyst-at-kantar-2694588069?refId=76RAULkUHnJbxypxLgrrkA%3D%3D&trackingId=gA6BK3vP%2FV9GKQu706Mxyw%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://es.linkedin.com/jobs/view/data-engineer-scala-ingl%C3%A9s-alto-at-cgi-2694583852?refId=76RAULkUHnJbxypxLgrrkA%3D%3D&trackingId=%2FtXJ%2B5rtl7lgVXrC2qG%2B8A%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card']
Job type ----
95
['Employment type\nFull-time', 'Employment type\nFull-time']
Job industry ----
['Industries\nMarket Research', 'Industries\nInformation Technology and Services']
Job seniority ----
['Seniority level\nAssociate', 'Seniority level

### 

In [54]:
dataframe_structure = {'Job Title': job_titles, 'Company Name':company_name, 'Location':job_location, 'Date Scraped':date_scraped, 'Job URL':links, 'Job Industry':job_industry, 'Job Type':job_type, 'Job Seniority':job_seniority, 'Job Description':job_desc}

In [55]:
try:
    last_index_in_csv_1 = last_index_in_csv+1
except:
    last_index_in_csv_1 = 0
df = pd.DataFrame(dataframe_structure)
df.index += last_index_in_csv_1

In [56]:
df['Job Industry'] = df['Job Industry'].replace(r'Industries\n','',regex=True)
df['Job Seniority'] = df['Job Seniority'].replace(r'Seniority level\n','',regex=True)
df['Job Description'] = df['Job Description'].replace(r'Job Description\n','',regex=True).replace(r'\n',' ',regex=True)
df['Job Type'] = df['Job Type'].replace(r'Employment Type\n','',regex=True)

In [57]:
df.to_csv('Linkedin_data_info_sp.csv',mode='a',header=False)